In [1]:
import datetime
from dateutil.relativedelta import relativedelta

import numpy as np
np.float = float    
np.int = int   #module 'numpy' has no attribute 'int'
np.object = object    #module 'numpy' has no attribute 'object'
np.bool = bool    
import pandas as pd 
import time
from selenium import webdriver 
from selenium.webdriver.support.ui import Select
from selenium.webdriver.firefox.options import Options 
from bs4 import BeautifulSoup
import re
from datetime import datetime,timedelta
from datetime import date
import openpyxl
from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows
import os.path
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import pyautogui as pag
import shutil
from os.path import exists
from selenium.webdriver.support.ui import Select


import bets_lib as bets

#Current bottlenckes:
#No Gold* filter
#Can only select as good as quarters

In [2]:
#Below Functions need driver to point to tournament list: https://be.tournamentsoftware.com/find?DateFilterType=0&StartDate=2024-08-20&EndDate=2024-11-21&page=1
def load_all_tournaments(driver):
    try:
        loadMoreButton = driver.find_element(by=By.ID, value = 'elem_loadmore')
        while True:
            loadMoreButton.click()
            time.sleep(1)
    except:
        wait = WebDriverWait(driver, 10)
        element = wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="searchResultArea"]')))

def select_level(driver,level):
    xpath = ''
    if level.lower() == 'international':
        xpath = '/html/body/div[1]/section/form/div/div/div[2]/div[2]/div/div/ul/li[4]/fieldset/ul/li[1]/div/label'
    if level.lower() == 'nationals':
        xpath = '//*[@id="filter_tournaments"]/div[2]/div/div/ul/li[4]/fieldset/ul/li[2]/div/label'
    if level.lower() == 'gold':
        xpath = '//*[@id="filter_tournaments"]/div[2]/div/div/ul/li[4]/fieldset/ul/li[3]/div/label'
    if level.lower() == 'silver':
        xpath = '//*[@id="filter_tournaments"]/div[2]/div/div/ul/li[4]/fieldset/ul/li[4]/div/label'
    if level.lower() == 'bronze':
        xpath = '//*[@id="filter_tournaments"]/div[2]/div/div/ul/li[4]/fieldset/ul/li[5]/div/label'
    if level.lower() == 'tier 4':
        xpath = '//*[@id="filter_tournaments"]/div[2]/div/div/ul/li[4]/fieldset/ul/li[6]/div/label'

    if not xpath:
        print(f'Couldnt find "{level}"')
        return 0

    selectedBox = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, xpath))
    )
    selectedBox.click()
    time.sleep(1)

def select_age(driver, age):
    ageGroupXpath = '//*[@id="TournamentExtendedFilter_AgeGroupID"]'
    ageSelector = Select(driver.find_element(by=By.XPATH,value = ageGroupXpath))
    # Print out all the options in the select dropdown
    options = ageSelector.options
    ageOptions = []
    for option in options[1:]:
        if not 'Masters' in option.text:
            ageOptions.append(option.text) 

    if age in str(ageOptions):
        ageSelector.select_by_visible_text(age)
    time.sleep(1)

def extract_links(driver):

    class Tournament():
        def __init__(self,title,link):
            self.name = title
            self.link = link
            self.startDate = ''
            self.level = ''

    linkRegex = re.compile('href=".*?"')
    titleRegex = re.compile('title=".*?"')
    source = driver.page_source
    soup = BeautifulSoup(source,features='html.parser') 
    tournamentsHtml = soup.find_all('h4',class_= 'media__title')


    tournaments = []
    for t in tournamentsHtml:
        tLink = re.findall(linkRegex,str(t))[0].replace('href=','').replace('"','')
        tLink = 'https://be.tournamentsoftware.com/tournament/' + tLink.split('=')[1]
        tTitle = re.findall(titleRegex,str(t))[0].replace('title=','').replace('"','')
        tournaments.append(Tournament(tTitle,tLink))
        
    return tournaments

def get_criteria():
    class Requirement():
        def __init__(self,age,level,minResult):
            self.level = level
            self.age = age
            self.winner = False
            
            if minResult.lower() == 'winner':
                self.minResult = ['Final']
                self.winner = True
            elif minResult.lower() == 'semis':
                self.minResult = ['Final','Semi final']
            elif minResult.lower() == 'quarters':
                self.minResult = ['Final','Semi final','Quarter final']

    requirements = []
    ## Tournament specific ##
    #U17 nationals - win
    #U19 Nationals - medal
    requirements.append(Requirement('U17','Nationals','Winner'))
    requirements.append(Requirement('U19','Nationals','Semis'))
    
    ## Level Specific ##
    #U19 Gold
    #Senior Gold
    requirements.append(Requirement('U19','Gold','Semis'))
    requirements.append(Requirement('Senior','Gold','Semis'))

    return requirements

def get_likely_candidates(driver,ageGroup, numPlayers=50):

    class TrackedPlayer():
        def __init__(self,name,pid):
            self.name = name
            self.id = pid
            self.event = 'ms'

    if ageGroup == 'U19':
        url = 'https://be.tournamentsoftware.com/ranking/category.aspx?id=41383&category=574&C574CS=10&C574FTYAF=0&C574FTYAT=0&C574FOG_2_F512=&p=1&ps=50'
    driver.get(url)
    div = driver.find_element(By.XPATH,'//*[@id="cphPage_cphPage_cphPage_C574CS_chosen"]')
    div.click()
    ActionChains(driver)\
                .move_to_element_with_offset(div,0,0)\
                .click()\
                .perform()
    pag.press('down', presses = how_many_presses(ageGroup))
    pag.press('enter')
    _filter = driver.find_element(By.XPATH,'//*[@id="cphPage_cphPage_cphPage_btnFilter"]')
    _filter.click()
    time.sleep(3)
    table = getRowsFromTableAsHTML(driver)
    usefulRegex = re.compile('&amp;player=[0-9]{1,14}">.*?<')
    profileRegex = re.compile('/player-profile.*?"')
    players = []
    for i,row in enumerate(table.rows):
        if ('noruler' in str(row)) or (i > numPlayers):
            break
        info = str(re.findall(usefulRegex,str(row))).split('>')
        name = info[1].replace('<','').replace("']",'')
        pLink = 'https://be.tournamentsoftware.com' + (re.findall(profileRegex,str(row))[0][:-1])
        driver.get(pLink)
        pid = (driver.find_element(By.XPATH,'/html/body/div[1]/div[3]/div[1]/div/div/div[2]/h2/span[2]').text)[1:-1]
        players.append(TrackedPlayer(name,pid)) 
    return players

def how_many_presses(age_group):
    if age_group == 'U10':
        return 1
    if age_group == 'U11':
        return 2
    if age_group == 'U12':
        return 3
    if age_group == 'U13':
        return 4
    if age_group == 'U14':
        return 5
    if age_group == 'U15':
        return 6
    if age_group == 'U16':
        return 7
    if age_group == 'U17':
        return 8
    if age_group == 'U18':
        return 9
    if age_group == 'U19':
        return 10
    if age_group == 'Senior':
        return 11
    return 0

def getRowsFromTableAsHTML(driver,xpath='//*[@id="content"]/table/tbody/tr[1]'):
    class Table():
        def __init__(self):
            self.headders = []
            self.rows = []

    #find initial tr[rowNum] from xpath
    table = Table()
    startRow = 1 
    lastRow = 'tr[1]'
    row = startRow
    rowXpath = xpath

    running = True
    getHeadders = True
    while running:
        rowXpath = rowXpath + '/td[0]'
        i = 0
        rowCells = []
        while True:
            try:
                i += 1
                rowXpath = rowXpath.replace(f'/td[{i-1}]',f'/td[{i}]')
                out = driver.find_element(By.XPATH,rowXpath)
                html = out.get_attribute('outerHTML')
                if not '<td>&nbsp;</td>' in html:
                    rowCells.append(html)
            except:
                break
        if len(rowCells) > 0:
            table.rows.append(rowCells)
        row += 1
        nextRow = f'tr[{row}]'
        rowXpath = xpath.replace(lastRow,nextRow)
        try:
            driver.find_element(By.XPATH,rowXpath)
        except:
            running = False
    return table





In [5]:
### Determine Tournaments to Check  ###

#get url to point to correct time period.
baselineUrl = 'https://be.tournamentsoftware.com/find?DateFilterType=0&StartDate=STARTDATE&EndDate=ENDDATE&page=1'
selectionDate = '2024-09-16'
selectionAge = 'U19'
checkFrom = selectionDate
checkUntilRaw = datetime.strptime(selectionDate,'%Y-%m-%d') - relativedelta(years=1)
checkUntil = checkUntilRaw.strftime('%Y-%m-%d')
tournamentsUrl = baselineUrl.replace('STARTDATE',checkUntil).replace('ENDDATE',checkFrom)

#load basic url
driver = webdriver.Firefox()
driver.get(tournamentsUrl)
bets.deal_with_cookies(driver)

likelyPlayers = get_likely_candidates(driver,selectionAge,20)

driver.get(tournamentsUrl)
 #Level
select_level(driver,'gold')
select_level(driver,'nationals')
 #Age group:
select_age(driver,'U19') 

load_all_tournaments(driver)

time.sleep(1)

#Extract links:
tournaments = extract_links(driver)
#Find results of first tournament
t = tournaments[2]
#change link to show matches tab
driver.get(t.link+'/Matches')
source = driver.page_source
soup = BeautifulSoup(source,features='html.parser') 
datesHtml = soup.find_all('a',class_= 'js-date-selection-tab')
dayLinks = []

#deal with tourneys spanning a few days
if datesHtml:
    dayLinkRegex = re.compile('data-href=".*?"')
    dayLinksRaw = re.findall(dayLinkRegex,str(datesHtml))
    for link in dayLinksRaw:
        dateNum = link.split('date=')[1].replace('"','')
        l = 'https://be.tournamentsoftware.com' + link.replace('data-href=','').replace('"','').split('/Matches/')[0] + '/matches/' + dateNum
        dayLinks.append(l)

#find tourney start date.
if dayLinks:
    driver.get(dayLinks[0])
    source = driver.page_source
    soup = BeautifulSoup(source,features='html.parser') 

startDateRaw = soup.find_all('span',class_ = 'module__title-sub')
startDate = str(startDateRaw).split('>')[1].split('<')[0]
dateObj = datetime.strptime(startDate, "%d %B %Y")
dateStr = datetime.strftime(dateObj,'%d/%m/%Y')
t.startDate = dateStr

tournamentLevel = driver.find_element(By.XPATH, value= '/html/body/div/div[3]/div[1]/div/div/div[2]/div/ul/li/span').text
t.level = tournamentLevel

driver.get(dayLinks[1]) #REMOVE THIS ---------------------------------------------------#######################################################################
source = driver.page_source #and this
soup = BeautifulSoup(source,features='html.parser') #this too

matchRaw = soup.find_all('div',class_ = 'match match--list')


In [8]:
uniqueMatches = str(matchRaw)[1:].split('<div class="match match--list">')
uniqueMatches.pop(0)

playerNameRegex = re.compile('player=\d{0,4}"><span class="nav-link__value">.*?</span>')
playerLinkRegex = re.compile('/sport.*?;player=\d{0,4}')
roundRegex = re.compile('title=.*?final')
eventLongRegex = re.compile('href=.*?</span>')
eventRegex = re.compile('value">.*?<')

wait = WebDriverWait(driver, 10)
class TournamentPlayer():
    def __init__(self,name,tLink,roundPlayed,result,event):
        self.name = (name.split('>')[2].replace('</span','')).split(' [')[0]
        self.link = 'https://be.tournamentsoftware.com/tournament/' + tLink.split('id=')[1].replace('&amp;','/').replace('=','/')
        self.id = ''
        self.roundPlayed = roundPlayed
        self.result = result
        self.event = event

criteria = get_criteria()
playerLongListRaw = []
for c in criteria:
    if (c.age.lower() in t.name.lower()) and (c.level.lower() in t.level.lower()): #Check for correct Tournament
        for m in uniqueMatches:
            for res in c.minResult: #Itterate through possible rounds
                if ('="' + res.lower()) in m.lower():
                    #Find Playernames, ID and who won
                    players = re.findall(playerNameRegex,m)
                    links = re.findall(playerLinkRegex,m)
                    matchHTML = m.split('match__row ')[1]
                    teamHTML = matchHTML.split('match__row"')
                    #Find round:
                    roundHTML = re.findall(roundRegex,m.lower())
                    roundPlayedString = str(roundHTML).split('"')[1]
                    roundPlayed = 0
                    if 'quarter' in roundPlayedString:
                        roundPlayed = 2
                    elif 'semi' in roundPlayedString:
                        roundPlayed = 1
                    #find event
                    eventLongHTML = re.findall(eventLongRegex,m)[0]
                    eventHTML = re.findall(eventRegex,str(eventLongHTML))
                    event = str(eventHTML).replace('value">','').replace('<','')
                    #change to Mens or Womans from Open or Boys/Girls
                    event = event.lower().split(' ')[0][2:].replace('os','ms').replace('od','md').replace('b','m').replace('g','w')
                    for i in range(0,len(players)):
                        for t1 in teamHTML:
                            if (players[i].lower() in t1.lower()) and ('has-won' in t1.lower()): 
                                playerLongListRaw.append(TournamentPlayer(players[i],links[i],roundPlayed,'won',event))
                            else:
                                playerLongListRaw.append(TournamentPlayer(players[i],links[i],roundPlayed,'lost',event))

playerLongList = []
idsLogged = set()
for p in playerLongListRaw:
    for lp in likelyPlayers:
        if p.name.lower() == lp.name.lower():
            driver.get(p.link)
            element = wait.until(EC.visibility_of_element_located((By.XPATH,'/html/body/div/div[4]/div/div[1]/div/div/div[2]/h4/span')))
            pid = driver.find_element(By.XPATH,'/html/body/div/div[4]/div/div[1]/div/div/div[2]/h4/span').text
            p.id = pid[1:-1]
        print(p.event,lp.event)
        if (p.id == lp.id) and (p.event == lp.event): # and p.id not in idsLogged:
            if p.id not in idsLogged:
                playerLongList.append(p)    
                idsLogged.add(p.id)
            else:
                for pInList in playerLongList:
                    if pInList.id == p.id and pInList.roundPlayed > p.roundPlayed:
                        pInList.roundPlayed = p.roundPlayed
                        pInList.result = p.result
                        #print()
                        #print(pInList.name,pInList.roundPlayed,pInList.result)
                        #print('swapped')
                        #print(p.name,p.roundPlayed,p.result) 

for p in playerLongList:
    print(p.name,p.id,p.roundPlayed,p.result,p.event) 


['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
['xd ms
